# AI YouTube Shorts Pipeline (Notebook)

This notebook automates: story generation (OpenAI), video generation (Pika/Runway), uploading to YouTube Shorts, and cleanup. It's designed to be run non-interactively (via GitHub Actions). Replace the environment variables in GitHub Secrets.

**Required secrets (set in GitHub repo Settings → Secrets):**
- `OPENAI_API_KEY`
- `PIKA_API_KEY` (or set `VIDEO_PROVIDER` to `runway` and provide `RUNWAY_API_KEY`)
- `YOUTUBE_CLIENT_ID`
- `YOUTUBE_CLIENT_SECRET`
- `YOUTUBE_REFRESH_TOKEN` (recommended)

Run this notebook locally first to verify before enabling Actions.

In [ ]:
import os
import json
from dotenv import load_dotenv

# Load .env locally — GitHub Actions will ignore this and use ENV variables
if os.path.exists(".env"):
    load_dotenv()

print("Environment loaded successfully.")


In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PIKA_API_KEY = os.getenv("PIKA_API_KEY")
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")

GOOGLE_CREDS = os.getenv("GOOGLE_CREDENTIALS")  # Full JSON string

# === CLEAN LOADING ===
GOOGLE_DATA = None
if GOOGLE_CREDS:
    # Print the first 100 characters of the raw string for final confirmation
    print(f"DEBUG: Raw GOOGLE_CREDS starts with: {GOOGLE_CREDS[:100]}")
    GOOGLE_DATA = json.loads(GOOGLE_CREDS.strip())
# =====================

print("Keys loaded:", "OPENAI" if OPENAI_API_KEY else None,
      "PIKA" if PIKA_API_KEY else None,
      "ELEVEN" if ELEVENLABS_API_KEY else None,
      "GOOGLE" if GOOGLE_DATA else None)

In [ ]:
import openai
import time

client = openai.OpenAI(api_key=OPENAI_API_KEY)

def generate_script(animal="baby penguin"):
    prompt = f"Write a 10-second cute scene involving a {animal}."
    max_retries = 5
    delay = 2 # initial delay in seconds

    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}]
            )
            script = response.choices[0].message.content
            print(f"Script generated successfully on attempt {attempt + 1}.")
            return script
        
        except openai.RateLimitError as e:
            if attempt < max_retries - 1:
                print(f"Rate limit exceeded (Attempt {attempt + 1}). Retrying in {delay} seconds...")
                time.sleep(delay)
                delay *= 2 # Exponential backoff
            else:
                # If all retries fail, raise the error
                raise e 
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            raise

script = generate_script()

In [ ]:
import requests
import time
import os

# Note: Using RUNWAYML_API_SECRET as per the previous fix, assuming it's loaded as RUNWAY_API_KEY locally
RUNWAY_API_KEY = os.getenv("RUNWAYML_API_SECRET") 

def create_video_runway_rest(prompt):
    print("Submitting to Runway Gen-2 (REST API)...")

    # FIX 1: Use the more robust 'jobs' endpoint
    base_url = "https://api.runwayml.com/v1"
    url = f"{base_url}/jobs" 

    headers = {
        "Authorization": f"Bearer {RUNWAY_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "gen-2",
        "prompt": prompt,
        "duration": 4,
        "seed": int(time.time())
    }

    # --- 1. Create generation (Job) ---
    r = requests.post(url, json=payload, headers=headers)
    
    # Check for submission errors
    r.raise_for_status() 
    
    data = r.json()
    
    # FIX 2: Job ID is returned as 'id'
    if 'id' not in data:
        print(f"ERROR: Runway submission failed. Response: {data}")
        raise ValueError("Runway job submission failed. Check API key/credits.")
        
    job_id = data["id"]
    print(f"Job created: {job_id}")

    # --- 2. Poll status ---
    status_url = f"{base_url}/jobs/{job_id}"

    while True:
        time.sleep(3) # Sleep first to avoid hammering the API instantly
        
        resp = requests.get(status_url, headers=headers)
        resp.raise_for_status()
        
        status_data = resp.json()
        state = status_data.get("status")

        print("Status:", state)

        if state == "succeeded":
            # FIX 3: Get URL from the job result
            # The URL is usually nested, but often available directly in the 'result' object
            video_url = status_data.get("result", {}).get("video_url")
            if not video_url:
                raise KeyError("Video URL not found in succeeded job result.")
            break

        if state == "failed":
            error_msg = status_data.get("error", "Unknown error")
            raise RuntimeError(f"Runway generation failed! Error: {error_msg}")

        # Continue polling if status is 'pending', 'processing', etc.

    # --- 3. Download video ---
    vid_file = "runway_video.mp4"
    r_video = requests.get(video_url)
    r_video.raise_for_status() # Check for download errors

    open(vid_file, "wb").write(r_video.content)

    print("Downloaded:", vid_file)
    return vid_file

# Execute the new function
# video_path = create_video_runway_rest(script) 
# video_path # Remember to update the calling line in your notebook

In [ ]:
import requests

def create_audio(text):
    url = "https://api.elevenlabs.io/v1/text-to-speech/EXAVITQu4vr4xnSDxMaL"  
    headers = {"xi-api-key": ELEVENLABS_API_KEY}
    payload = {"text": text, "voice_settings": {"stability": 0.3}}

    audio = requests.post(url, json=payload, headers=headers)
    path = "audio.mp3"
    open(path, "wb").write(audio.content)
    return path

audio_path = create_audio(script)
audio_path


In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip

output_file = "final_video.mp4"

video = VideoFileClip(video_path)
audio = AudioFileClip(audio_path)

final = video.set_audio(audio)
final.write_videofile(output_file)

output_file


In [ ]:
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

def upload_youtube(video_file, title="Cute Animal Short", desc="Generated by AI"):
    creds = Credentials.from_authorized_user_info(GOOGLE_DATA["installed"])

    youtube = build("youtube", "v3", credentials=creds)

    request = youtube.videos().insert(
        part="snippet,status",
        body={
            "snippet": {"title": title, "description": desc, "categoryId": "15"},
            "status": {"privacyStatus": "public"}
        },
        media_body=MediaFileUpload(video_file)
    )
    response = request.execute()
    print("Uploaded:", response["id"])
    return response["id"]

yt_id = upload_youtube("final_video.mp4")
yt_id


In [ ]:
print("Pipeline finished successfully!")
print("YouTube Video ID:", yt_id)
